In [ ]:
from os import getenv
from connect_db import connect_db
con = connect_db(
    username=getenv("ORACLE_USERNAME"),
    password=getenv("ORACLE_PASSWORD"),
    host=getenv("ORACLE_HOST"),
    port=int(getenv("ORACLE_PORT"))
)
cur = con.cursor()

In [ ]:
import numpy as np

def deviate(n: int, error: float) -> int:
    deviation = int(n * error)
    return n + np.random.randint(-deviation, deviation + 1)

AVERAGE_KILOMETER_PER_HOURS = 580
def get_duration_in_hours(kilometers: int) -> float:
    return deviate(kilometers, 0.01) / AVERAGE_KILOMETER_PER_HOURS

In [ ]:
from typing import NamedTuple
from datetime import time

class Schedule(NamedTuple):
    route_id: str
    time_of_day: time

WEEKLY_SCHEDULE: dict[int, list[Schedule]] = {
    0: [
        Schedule("SFS0010", time(8, 0)),
        Schedule("SFS0104", time(9, 0)),
        Schedule("SFS0011", time(12, 30)),
        Schedule("SFS0105", time(14, 30)),
        Schedule("SFS0010", time(15, 30)),
        Schedule("SFS0011", time(18, 0))
    ],
    1: [
        Schedule("SFS0115", time(8, 0)),
        Schedule("SFS0100", time(9, 0)),
        Schedule("SFS0101", time(15, 0)),
        Schedule("SFS0100", time(20, 0)),
        Schedule("SFS0100", time(23, 30))
    ],
    2: [
        Schedule("SFS0302", time(3, 0)),
        Schedule("SFS0300", time(4, 0)),
        Schedule("SFS0105", time(9, 30)),
        Schedule("SFS0104", time(14, 0))
    ],
    3: [
        Schedule("SFS0301", time(7, 0)),
        Schedule("SFS0010", time(8, 30)),
        Schedule("SFS0303", time(8, 30)),
        Schedule("SFS0011", time(12, 0)),
        Schedule("SFS0010", time(15, 30)),
        Schedule("SFS0011", time(18, 0)),
        Schedule("SFS0116", time(23, 30))
    ],
    4: [
        Schedule("SFS0104", time(9, 0)),
        Schedule("SFS0201", time(11, 30)),
        Schedule("SFS0105", time(14, 0)),
        Schedule("SFS0205", time(15, 0))
    ],
    5: [
        Schedule("SFS0206", time(7, 30)),
        Schedule("SFS0111", time(8, 30)),
        Schedule("SFS0202", time(20, 30)),
        Schedule("SFS0112", time(21, 0))
    ],
    6: [
        Schedule("SFS0105", time(9, 0)),
        Schedule("SFS0013", time(10, 0)),
        Schedule("SFS0104", time(14, 0)),
        Schedule("SFS0014", time(18, 0)),
    ]
}

In [ ]:
DISTANCE: dict[str, int] = {}

stmt = "SELECT ROUTE_ID, DISTANCE_IN_KM FROM ROUTE"
for row in cur.execute(stmt):
    DISTANCE[row[0]] = row[1]

In [ ]:
from datetime import datetime
 
class Info(NamedTuple):
    can_use_after: datetime
    airport: str
    
# very sloppy aircraft management
AIRCRAFT: dict[str, Info] = {}
stmt = "SELECT AIRCRAFT_ID FROM AIRCRAFT"
for row in cur.execute(stmt):
    AIRCRAFT[row[0]] = Info(datetime.min, "KUL")
    
def choose_aircraft(dt: datetime, start_airport: str) -> str:
    sorted_aircrafts = sorted(AIRCRAFT.items(), key=lambda t: t[1].can_use_after)
    
    for aircraft, info in sorted_aircrafts:
        if dt > info.can_use_after and start_airport == info.airport:
            return aircraft
    
    for aircraft, info in sorted_aircrafts:
        if dt > info.can_use_after:
            return aircraft
        
    return sorted_aircrafts[0][0]
    
def use_aircraft(aircraft_id: str, dt: datetime, end_airport: str):
    AIRCRAFT[aircraft_id] = Info(dt, end_airport)

In [ ]:
START_AND_END_AIRPORTS: dict[str, tuple[str, str]] = {}

stmt = "SELECT ROUTE_ID, FROM_AIRPORT_ID, TO_AIRPORT_ID FROM ROUTE"
for row in cur.execute(stmt):
    START_AND_END_AIRPORTS[row[0]] = row[1], row[2]

In [ ]:
from datetime import timedelta

starting_date = datetime(2023, 1, 1)
now = datetime.now()

sql = "    INTO FLIGHT (ROUTE_ID, AIRCRAFT_ID, DEPARTURE_DATETIME, EST_DURATION_IN_HOUR) VALUES ('{}', '{}', TO_TIMESTAMP('{:%Y-%m-%d %H:%M:%S}', 'YYYY-MM-DD HH24:MI:SS'), {:.2f})"
while starting_date < now:
    schedule = WEEKLY_SCHEDULE[starting_date.weekday()]
    for flight in schedule:
        route_id = flight.route_id
        departure_time = datetime.combine(starting_date.date(), flight.time_of_day)
        start_airport, end_airport = START_AND_END_AIRPORTS[route_id]
        aircraft_id = choose_aircraft(departure_time, start_airport)
        est_duration = get_duration_in_hours(DISTANCE[route_id])
        
        print(sql.format(route_id, aircraft_id, departure_time, est_duration))
        
        use_aircraft(aircraft_id, departure_time + timedelta(hours=est_duration + 0.5), end_airport)
    
    starting_date += timedelta(days=1)

In [ ]:
con.close()